In [50]:
#Import Packages
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import warnings
import nltk
import string
import seaborn as sns
from nltk.stem.porter import * 
from wordcloud import WordCloud

warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier



In [2]:
pwd

'C:\\Users\\Vardhaman\\Desktop\\DS\\Hackathon_3\\git_sentiment\\Notebooks'

In [17]:
df = pd.read_csv('../data/iter/rTrain.csv',index_col=0).dropna()
train = df[['tweet_id','lemma_tweet','sentiment']]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
train_vector = cv.fit_transform(train['lemma_tweet'].dropna()).toarray()

In [13]:
# stantdard scalar
# train_vector_scalar = StandardScaler(with_mean=False).fit_transform(train_vector)

In [19]:
X_train,X_test,y_train,y_test = train_test_split(train_vector_scalar,train['sentiment'],test_size=0.3,random_state=42)

In [38]:
# X_test


In [39]:
# cdf
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# norma = Normalizer()
# X_train = norma.fit_transform(X_train)
# X_test=  norma.transform(X_test)

# lsa = NMF(n_components=100)
# mnb = MultinomialNB()

# # train_text = vectorizer.fit_transform(raw_text_train)
# X_train = lsa.fit_transform(X_train)
# X_train = Normalizer(copy=False).fit_transform(X_train)
# X_test = lsa.transform(X_test)
# X_test = Normalizer(copy=False).transform(X_test)
# mnb.fit(X_train,y_train)

In [40]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import classification_report,confusion_matrix,f1_score

# nb = MultinomialNB(alpha=0.01)
# nb.fit(X_train,y_train)
# y_pred = nb.predict(X_test)
# # print(confusion_matrix(y_test,y_pred))   
# # print(classification_report(y_test,y_pred))   
# f1_score(y_test,y_pred,average='weighted')

In [42]:
randomF = RandomForestClassifier()
randomF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
y_pred = randomF.predict(X_test)

In [49]:
f1_score(y_pred,y_test,average='weighted')

0.6255960496649039

In [51]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [52]:
y_pred_xg = xgb.predict(X_test)

In [55]:
f1_score(y_pred_xg,y_test,average='weighted')

0.7057583822296658

# Test

In [9]:
test = pd.read_csv('../data/raw/test.csv')

In [48]:
# StopWord Removal

#import nltk
#nltk.download('stopwords')

import string
import re
import nltk
 
from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')
 
from nltk.tokenize import TweetTokenizer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textblob import TextBlob
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    nltk_tagged = nltk.pos_tag(tokenizer.tokenize(sentence))  
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
          if (word not in stopwords_english and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
                #tweet = str(TextBlob(word).correct())                
                if tag is None:
                    #if there is no available tag, append the token as is
                    lemmatized_sentence.append(word)
                else:        
                    #else use the tag to lemmatize the token
                    lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)    
    
def clean_tweets(tweet):    
    # remove numbers
    tweet = tweet.lower()
    tweet = re.sub(r'\d+', '', tweet)
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^rt[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    return lemmatize_sentence(tweet)

In [50]:
rtest = test.copy()
rtest['lemma_tweet'] = rtest.apply(lambda row: clean_tweets(str(row['tweet'])), axis=1)

In [51]:
# rTrain = cTrain.copy()

freq = pd.Series(' '.join(rtest['lemma_tweet']).split()).value_counts()[:10]
freq
freq = list(freq.index)
rtest['lemma_tweet'] = rtest['lemma_tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

freq = pd.Series(' '.join(rtest['lemma_tweet']).split()).value_counts()[-10:]
freq
freq = list(freq.index)
rtest['lemma_tweet'] = rtest['lemma_tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
rtest.head()

# rTrain.to_csv('../data/iter1/rTrain.csv') 



,tweet_id,tweet,lemma_tweet
0,7506,Audience Q: What prototyping tools do you use?...,audience q prototyping tool use sketchbook sha...
1,7992,At SXSW? Send Your Best Photos &amp; Videos to...,send best photo amp video citizen journalism c...
2,247,@mention and here's a pic of you winning your...,pic win unsix cc cont
3,7688,Google Marissa Mayer: mobile phone as a cursor...,marissa mayer mobile phone cursor physical loc...
4,3294,#SXSW Google maps is even cooler than I thought,map even cool think


In [57]:
train_vector = cv.fit_transform(train['lemma_tweet'].dropna()).toarray()
test_vector = cv.transform(rtest['lemma_tweet'].dropna()).toarray()

In [62]:
nb.fit(train_vector,train['sentiment'])
tweety_pred = pd.Series(nb.predict(test_vector))

In [67]:
pred_df = pd.concat([test['tweet_id'],tweety_pred],axis=1).set_index('tweet_id')
pred_df.columns = ['sentiment']

In [69]:
pred_df.to_csv(r'C:\Data Science\Jupyter Notebook\GitHackathon\result_submissions\tweety_cv.csv')

In [70]:
from sklearn.linear_model import LogisticRegression

In [71]:
lr = LogisticRegression()